In [1]:
import cv2
import numpy as np
from PIL import Image
import rospy
from sensor_msgs.msg import Image, PointCloud2  # Import PointCloud2 message type
import ros_numpy
import sensor_msgs.point_cloud2 as pc2
import struct
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

In [2]:

width, height = 320, 180

In [5]:
#!/usr/bin/env python
def get_pixel_rgb(image_msg, x, y):
    if x < 0 or y < 0 or x >= width or y >= height:
        rospy.logerr("Requested pixel coordinates are out of image bounds")
        return None
    
    byte_index = y * image_msg.step + x * 3
    try:
        r = image_msg.data[byte_index]
        g = image_msg.data[byte_index + 1]
        b = image_msg.data[byte_index + 2]
        return (r, g, b)
    except IndexError:
        rospy.logerr("Index out of range while accessing image data")
        return None

# Define height and width as global variables
height = 180
width = 320
two_d_array = np.zeros((height, width, 3), dtype=np.uint8)
def image_segmented_callback(data):
    global height, width, two_d_array
    # Convert ROS image message to OpenCV image
    bridge = CvBridge()
    image = bridge.imgmsg_to_cv2(data, desired_encoding="passthrough")
    # Get height and width of the image
    rospy.loginfo("Received segmented image with height: {} and width: {}".format(height, width))
    # Define the desired width and height
    # Resize the image
    resized_image = cv2.resize(image, (width, height))
    for i in range(height):
        for j in range(width):
            two_d_array[i][j] = resized_image[i][j]

def float32_to_binary(float_value):
    # Convert float value to binary representation
    binary_representation = struct.pack('f', float_value)
    # Convert binary representation to binary string
    binary_string = ''.join(f'{byte:08b}' for byte in binary_representation)
    return binary_string

def point_cloud_callback(msg):
    global two_d_array
    rospy.loginfo(msg.height)
    rospy.loginfo(msg.width)
    new_pt = []
    height = msg.height
    width = msg.width
    rospy.loginfo(msg.point_step)
    rospy.loginfo(msg.row_step)
    rospy.loginfo((msg.fields[3].datatype))
    new_pt=[]
    rospy.loginfo("converting to numpy")
    pc_np=ros_numpy.numpify(msg)
    rgbsp=ros_numpy.point_cloud2.split_rgb_field(pc_np)
    rospy.loginfo("converted to numpy")
    rospy.loginfo(pc_np.shape)
    k=0
    for i in range(height):
        for j in range(width):
            #rospy.loginfo(rgbsp[i][j]['r'])
            #rospy.loginfo(rgbsp[i][j]['g'])
            #rospy.loginfo(rgbsp[i][j]['b'])
            rgbsp[i][j]['r']=two_d_array[i][j][2]
            rgbsp[i][j]['g']=two_d_array[i][j][1]
            rgbsp[i][j]['b']=two_d_array[i][j][0]
            k=k+1
            #two_d_array[i][j] = resized_image[i][j]
    pc_np=ros_numpy.point_cloud2.merge_rgb_fields(rgbsp)

    modified_msg = ros_numpy.point_cloud2.array_to_pointcloud2	(pc_np,stamp = None,frame_id = 'camera_color_optical_frame' )	
    pub.publish(modified_msg)
    rospy.loginfo(modified_msg.height)
    rospy.loginfo(modified_msg.width)

    
def main():
    rospy.init_node("point_cloud_modifier0")
    two_d_array = []
    rospy.Subscriber("/image_segmented", Image, image_segmented_callback)
    rospy.Subscriber("/voxel_cloud", PointCloud2, lambda msg: point_cloud_callback(msg))
    global pub
    pub = rospy.Publisher("/output/modified_point_cloud_topic", PointCloud2, queue_size=10)
    rospy.spin()

if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass


Binary representation of 3.14 as float32: 11000011111101010100100001000000
[INFO] [1714481752.908326]: Received segmented image with height: 180 and width: 320
[INFO] [1714481753.102547]: Received segmented image with height: 180 and width: 320
[INFO] [1714481753.108448]: 180
[INFO] [1714481753.110222]: 320
[INFO] [1714481753.112713]: 32
[INFO] [1714481753.113827]: 10240
[INFO] [1714481753.115171]: 7
[INFO] [1714481753.116496]: converting to numpy
[INFO] [1714481753.126148]: converted to numpy
[INFO] [1714481753.140307]: (180, 320)
[INFO] [1714481753.366491]: Received segmented image with height: 180 and width: 320
[INFO] [1714481753.625976]: Received segmented image with height: 180 and width: 320
[INFO] [1714481753.880636]: Received segmented image with height: 180 and width: 320
[INFO] [1714481754.020892]: 180
[INFO] [1714481754.022815]: 320
[INFO] [1714481754.029824]: 180
[INFO] [1714481754.032892]: 320
[INFO] [1714481754.034931]: 32
[INFO] [1714481754.036452]: 10240
[INFO] [171448